In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

In [2]:
data = pd.read_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5.csv')
data
# 카페 데이터만
data = data.loc[data['상권업종중분류명'] == '커피점/카페']  
data

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도,역명,거리,카카오url
7,60,28490990,406번째스토브,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,158050,7999.0,NaN,NaN,NaN,126.874168,37.525177,5호선 오목교(목동운동장앞),42.783155,NaN
8,81,28491095,우성커피숍,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,158090,8029.0,NaN,1,NaN,126.842935,37.522055,5호선 까치산,794.156052,NaN
10,96,23260840,버블베어,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,157788,7509.0,NaN,1,NaN,126.813358,37.580941,5호선 방화,261.092137,NaN
55,388,25337952,리버스로스터,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,158090,7909.0,NaN,1,NaN,126.823946,37.535437,5호선 마곡,1658.491978,NaN
61,406,27535314,필링,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,110320,3139.0,NaN,1,하,126.989443,37.570993,5호선 종로3가(탑골공원),125.480924,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20025,135714,17207878,세라젬웰카페,영등포점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,150030,7301.0,NaN,1,NaN,126.904491,37.519552,5호선 영등포시장,215.460909,NaN
20026,135716,17207899,미타임,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,133783,4806.0,NaN,1,NaN,127.057222,37.562579,5호선 답십리,587.455177,NaN
20027,135717,17207763,휴식상점,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,130843,2634.0,NaN,1,3,127.062036,37.563239,5호선 장한평,290.438891,NaN
20043,135797,17211625,단백한하우스,NaN,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,...,138812,5782.0,NaN,1,101,127.134560,37.493511,5호선 개롱,357.124389,NaN


In [3]:
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리","카카오url"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,406번째스토브,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 329-6,5호선 오목교(목동운동장앞),42.783155,NaN,오목교,오목교역 406번째스토브
8,우성커피숍,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 48,5호선 까치산,794.156052,NaN,까치산,까치산역 우성커피숍
10,버블베어,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 금낭화로 167,5호선 방화,261.092137,NaN,방화,방화역 버블베어
55,리버스로스터,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 가로공원로 71,5호선 마곡,1658.491978,NaN,마곡,마곡역 리버스로스터
61,필링,커피점/카페,커피전문점/카페/다방,서울특별시 종로구 수표로 110,5호선 종로3가(탑골공원),125.480924,NaN,종로3가,종로3가역 필링
...,...,...,...,...,...,...,...,...,...
20025,세라젬웰카페,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 영등포로 210,5호선 영등포시장,215.460909,NaN,영등포시장,영등포시장역 세라젬웰카페
20026,미타임,커피점/카페,커피전문점/카페/다방,서울특별시 성동구 천호대로 346,5호선 답십리,587.455177,NaN,답십리,답십리역 미타임
20027,휴식상점,커피점/카페,커피전문점/카페/다방,서울특별시 동대문구 한천로6길 47-2,5호선 장한평,290.438891,NaN,장한평,장한평역 휴식상점
20043,단백한하우스,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 문정로11길 12,5호선 개롱,357.124389,NaN,개롱,개롱역 단백한하우스


In [4]:
items = data["검색어"][263:]
items

1956           마포역 쥬씨도화점
1961     답십리역 떼루와떼루와답십리점
1962          상일동역 마이쥬카페
1968        명일역 르하임스터디카페
1969          답십리역 세븐일레븐
              ...       
20025      영등포시장역 세라젬웰카페
20026           답십리역 미타임
20027          장한평역 휴식상점
20043         개롱역 단백한하우스
20044           오금역 만랩커피
Name: 검색어, Length: 2858, dtype: object

In [5]:
rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
img_elem = pd.DataFrame(columns = ["ItemID","img_url"])
review_elem = pd.DataFrame(columns = ["ItemID", "review"])
user_review_id = {}
review_json = {}
image_json = {}

In [6]:
# 드라이버 구동
driver = webdriver.Chrome(executable_path='C:/Users/정예림/Desktop/추천시스템 프로젝트/chromedriver.exe')
# 창 크기 최대화
driver.maximize_window() 



count = 0
current = 0
indexing = 0
goal = len(items)
for item in items :
    current += 1
    print('진행상황 : ', current,'/',goal,sep="")
    # 리뷰가 없을 때의 코드
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    time.sleep(2)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)
    
    if len(driver.find_elements_by_xpath("//a[@class='moreview']")) != 0:
        print('식당 존재')
        driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
        reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
        time.sleep(2)
        content_url = reviewbutton.get_attribute("href") 
        #상세보기 URL수집
        url = driver.find_element_by_xpath('//*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]').get_attribute('href')
        data.iloc[indexing,-3] = url
        print(url)
        tabs = driver.window_handles
        driver.switch_to_window(tabs[1]) # 새 탭으로 이동
        driver.get(content_url) # 링크 열기
        time.sleep(3)
        html = driver.page_source
        time.sleep(20)
        soup = BeautifulSoup(html, 'html.parser')
        review_lists = soup.select('.list_evaluation > li')
        print(len(review_lists))
        if len(review_lists) != 0 :
            for i, review in enumerate(review_lists) :
                user_review = review.select('.txt_comment > span') # 리뷰
                rating = review.select('.grade_star > em') # 별점
                img_url = review.select('.link_photo > img') #이미지url
                user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                timestamp = review.select(' div > span.time_write') #시간정보
                try:
                    row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_ur}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)

                    time.sleep(1)
                except:
                    row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                    row = pd.DataFrame(row, index=[i])
                    rating_df = rating_df.append(row,ignore_index=True)
                    review_row = {"ItemID" : item, "review" : user_review[0].text}
                    review_row = pd.DataFrame(review_row, index=[i])
                    review_elem = review_elem.append(review_row, ignore_index = True)
                    try :
                        img_row = {"ItemID" : item, "img_url" : img_url}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    except :
                        img_row = {"ItemID" : item, "img_url" : None}
                        img_row = pd.DataFrame(img_row, index=[i])
                        img_elem = img_elem.append(img_row, ignore_index=True)
                    time.sleep(1)

                
                    
        else :
            print("리뷰가 없습니다")
            
        try:
            for i in range(2,500):
                time.sleep(3)
                another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
                another_review.click()
                time.sleep(3)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                review_lists = soup.select('.list_evaluation > li')
                if len(review_lists) != 0 :
                    for i, review in enumerate(review_lists) :
                        user_review = review.select('.txt_comment > span') # 리뷰
                        rating = review.select('.grade_star > em') # 별점
                        img_url = review.select('.link_photo > img') #이미지url
                        user_id = review.select('.append_item > a[data-userid]') # user 정보 html 파싱
                        timestamp = review.select(' div > span.time_write') #시간정보
                        try:
                            row = {"UserID":user_id[0].get('data-userid'),"ItemID":item,"Rating":rating[0].text,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                            
                        except:
                            row = {"UserID":None,"ItemID":item,"Rating":None,"Timestamp":timestamp[0].text}
                            row = pd.DataFrame(row, index=[i])
                            rating_df = rating_df.append(row,ignore_index=True)
                            review_row = {"ItemID" : item, "review" : user_review[0].text}
                            review_row = pd.DataFrame(review_row, index=[i])
                            review_elem = review_elem.append(review_row, ignore_index = True)
                            try :
                                img_row = {"ItemID" : item, "img_url" : img_url}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            except :
                                img_row = {"ItemID" : item, "img_url" : None}
                                img_row = pd.DataFrame(img_row, index=[i])
                                img_elem = img_elem.append(img_row, ignore_index=True)
                            time.sleep(1)
                        
                        
                            
        except:
            print("더 이상 리뷰 존재 X")
            driver.close()
        driver.switch_to_window(tabs[0])
        print("기본 페이지로 돌아가자")
            
    else:
        data.iloc[indexing,-3] = None
        print("식당 존재 x")
    
    indexing += 1


진행상황 : 1/2858
식당 존재 x
진행상황 : 2/2858
식당 존재
https://place.map.kakao.com/1767318696
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 3/2858
식당 존재 x
진행상황 : 4/2858
식당 존재
https://place.map.kakao.com/1192328262
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 5/2858
식당 존재
https://place.map.kakao.com/13056584
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 6/2858
식당 존재
https://place.map.kakao.com/1328088838
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 7/2858
식당 존재
https://place.map.kakao.com/8328655
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 8/2858
식당 존재
https://place.map.kakao.com/24765614
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 9/2858
식당 존재 x
진행상황 : 10/2858
식당 존재 x
진행상황 : 11/2858
식당 존재
https://place.map.kakao.com/26463339
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 12/2858
식당 존재
https://place.map.kakao.com/16500364
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 13/2858
식당 존재 x
진행상황 : 14/2858
식당 존재 x
진행상황 : 15/2858
식당 존재 x
진행상황 : 16/2858
식당 존재
https://place.map.kakao.com/20942331
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 17/2858
식당 존재
https://plac

식당 존재
https://place.map.kakao.com/835692943
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 119/2858
식당 존재
https://place.map.kakao.com/486782235
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 120/2858
식당 존재 x
진행상황 : 121/2858
식당 존재
https://place.map.kakao.com/10830319
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 122/2858
식당 존재
https://place.map.kakao.com/1181667709
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 123/2858
식당 존재
https://place.map.kakao.com/2078910428
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 124/2858
식당 존재
https://place.map.kakao.com/98496579
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 125/2858
식당 존재
https://place.map.kakao.com/19381446
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 126/2858
식당 존재
https://place.map.kakao.com/1368324452
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 127/2858
식당 존재
https://place.map.kakao.com/1347304742
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 128/2858
식당 존재
https://place.map.kakao.com/757366145
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 129/2858
식당 존재
https://place.map.kakao.com/228939039
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 

식당 존재 x
진행상황 : 222/2858
식당 존재
https://place.map.kakao.com/1111976054
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 223/2858
식당 존재
https://place.map.kakao.com/806305387
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 224/2858
식당 존재 x
진행상황 : 225/2858
식당 존재
https://place.map.kakao.com/744521648
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 226/2858
식당 존재
https://place.map.kakao.com/1704633111
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 227/2858
식당 존재
https://place.map.kakao.com/26909836
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 228/2858
식당 존재
https://place.map.kakao.com/236982649
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 229/2858
식당 존재
https://place.map.kakao.com/27542749
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 230/2858
식당 존재
https://place.map.kakao.com/409589446
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 231/2858
식당 존재
https://place.map.kakao.com/1390779926
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 232/2858
식당 존재
https://place.map.kakao.com/321907882
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 233/2858
식당 존재
https://place.map.kakao.com/1878693468
5
더 이상 리뷰 존재 X
기본

식당 존재
https://place.map.kakao.com/1515703673
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 336/2858
식당 존재
https://place.map.kakao.com/27306515
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 337/2858
식당 존재
https://place.map.kakao.com/1355312150
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 338/2858
식당 존재
https://place.map.kakao.com/1301801275
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 339/2858
식당 존재
https://place.map.kakao.com/1933396119
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 340/2858
식당 존재 x
진행상황 : 341/2858
식당 존재
https://place.map.kakao.com/1200840308
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 342/2858
식당 존재
https://place.map.kakao.com/1460704646
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 343/2858
식당 존재
https://place.map.kakao.com/21458966
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 344/2858
식당 존재
https://place.map.kakao.com/14101953
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 345/2858
식당 존재 x
진행상황 : 346/2858
식당 존재
https://place.map.kakao.com/1598656879
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 347/2858
식당 존재
https://place.map.kakao.com/1462413

식당 존재
https://place.map.kakao.com/26646865
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 477/2858
식당 존재
https://place.map.kakao.com/1598656879
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 478/2858
식당 존재
https://place.map.kakao.com/21304331
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 479/2858
식당 존재 x
진행상황 : 480/2858
식당 존재 x
진행상황 : 481/2858
식당 존재
https://place.map.kakao.com/1936960218
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 482/2858
식당 존재
https://place.map.kakao.com/14521437
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 483/2858
식당 존재 x
진행상황 : 484/2858
식당 존재
https://place.map.kakao.com/1445516655
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 485/2858
식당 존재 x
진행상황 : 486/2858
식당 존재 x
진행상황 : 487/2858
식당 존재
https://place.map.kakao.com/249179932
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 488/2858
식당 존재
https://place.map.kakao.com/708639293
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 489/2858
식당 존재
https://place.map.kakao.com/1154783384
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 490/2858
식당 존재 x
진행상황 : 491/2858
식당 존재 x
진행상황 : 492/2858
식당 존재
https://place.map.kakao.

...

더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1831/2858
식당 존재
https://place.map.kakao.com/16329997
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1832/2858
식당 존재
https://place.map.kakao.com/15925630
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1833/2858
식당 존재 x
진행상황 : 1834/2858
식당 존재 x
진행상황 : 1835/2858
식당 존재 x
진행상황 : 1836/2858
식당 존재 x
진행상황 : 1837/2858
식당 존재
https://place.map.kakao.com/24923499
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1838/2858
식당 존재
https://place.map.kakao.com/26853353
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1839/2858
식당 존재
https://place.map.kakao.com/26430433
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1840/2858
식당 존재
https://place.map.kakao.com/21396150
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1841/2858
식당 존재
https://place.map.kakao.com/2016738578
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1842/2858
식당 존재
https://place.map.kakao.com/27513379
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1843/2858
식당 존재
https://place.map.kakao.com/23612932
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1844/2858
식당 존재
https://place.map.kakao.com/1572966029


식당 존재
https://place.map.kakao.com/10217237
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1944/2858
식당 존재
https://place.map.kakao.com/16767863
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1945/2858
식당 존재 x
진행상황 : 1946/2858
식당 존재 x
진행상황 : 1947/2858
식당 존재
https://place.map.kakao.com/24910232
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1948/2858
식당 존재 x
진행상황 : 1949/2858
식당 존재 x
진행상황 : 1950/2858
식당 존재 x
진행상황 : 1951/2858
식당 존재 x
진행상황 : 1952/2858
식당 존재
https://place.map.kakao.com/18409983
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1953/2858
식당 존재 x
진행상황 : 1954/2858
식당 존재
https://place.map.kakao.com/22318790
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1955/2858
식당 존재 x
진행상황 : 1956/2858
식당 존재
https://place.map.kakao.com/1528592090
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1957/2858
식당 존재 x
진행상황 : 1958/2858
식당 존재 x
진행상황 : 1959/2858
식당 존재
https://place.map.kakao.com/18767387
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1960/2858
식당 존재
https://place.map.kakao.com/271223969
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 1961/2858
식당 존재 x
진행상황 : 1962/2858


https://place.map.kakao.com/26477284
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2074/2858
식당 존재
https://place.map.kakao.com/132668771
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2075/2858
식당 존재
https://place.map.kakao.com/440142287
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2076/2858
식당 존재
https://place.map.kakao.com/1126730091
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2077/2858
식당 존재 x
진행상황 : 2078/2858
식당 존재 x
진행상황 : 2079/2858
식당 존재 x
진행상황 : 2080/2858
식당 존재
https://place.map.kakao.com/830148993
2
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2081/2858
식당 존재
https://place.map.kakao.com/1943506703
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2082/2858
식당 존재 x
진행상황 : 2083/2858
식당 존재 x
진행상황 : 2084/2858
식당 존재
https://place.map.kakao.com/1642746873
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2085/2858
식당 존재
https://place.map.kakao.com/1637219532
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2086/2858
식당 존재
https://place.map.kakao.com/1205660091
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2087/2858
식당 존재
https://place.map.kakao.com/11882906
3
더 이상

0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2184/2858
식당 존재 x
진행상황 : 2185/2858
식당 존재
https://place.map.kakao.com/11831443
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2186/2858
식당 존재
https://place.map.kakao.com/8115422
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2187/2858
식당 존재
https://place.map.kakao.com/1292261866
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2188/2858
식당 존재
https://place.map.kakao.com/2096118235
1
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2189/2858
식당 존재
https://place.map.kakao.com/1018547774
4
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2190/2858
식당 존재
https://place.map.kakao.com/2075891233
3
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2191/2858
식당 존재
https://place.map.kakao.com/26617152
5
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2192/2858
식당 존재
https://place.map.kakao.com/602519384
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2193/2858
식당 존재 x
진행상황 : 2194/2858
식당 존재
https://place.map.kakao.com/2001001561
0
리뷰가 없습니다
더 이상 리뷰 존재 X
기본 페이지로 돌아가자
진행상황 : 2195/2858
식당 존재
https://place.map.kakao.com/23640936
0
리뷰가 없습니다
더 이상 리뷰 존재 X


KeyboardInterrupt: 

In [7]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,cj0olb,양평역 매머드커피익스프레스,2점,2019.12.01.
1,mlm7f0,양평역 매머드커피익스프레스,5점,2019.10.20.
2,vdui22,양평역 매머드커피익스프레스,5점,2019.08.26.
3,u9keii,양평역 매머드커피익스프레스,4점,2019.08.23.
4,vaui89,마포역 일야에스프레소,1점,2021.05.05.
...,...,...,...,...
7640,None,화곡역 메리하하,None,2018.12.12.
7641,,화곡역 메리하하,5점,2018.08.10.
7642,None,굽은다리역 카페1046,None,2016.12.15.
7643,None,신정역 카페펀코,None,2016.06.28.


In [9]:
img_elem

,ItemID,img_url
0,양평역 매머드커피익스프레스,None
1,양평역 매머드커피익스프레스,None
2,양평역 매머드커피익스프레스,None
3,양평역 매머드커피익스프레스,None
4,마포역 일야에스프레소,None
...,...,...
7640,화곡역 메리하하,None
7641,화곡역 메리하하,None
7642,굽은다리역 카페1046,None
7643,신정역 카페펀코,None


In [10]:
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,카카오url,역,검색어
7,406번째스토브,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 329-6,5호선 오목교(목동운동장앞),42.783155,NaN,오목교,오목교역 406번째스토브
8,우성커피숍,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 오목로 48,5호선 까치산,794.156052,https://place.map.kakao.com/1767318696,까치산,까치산역 우성커피숍
10,버블베어,커피점/카페,커피전문점/카페/다방,서울특별시 강서구 금낭화로 167,5호선 방화,261.092137,None,방화,방화역 버블베어
55,리버스로스터,커피점/카페,커피전문점/카페/다방,서울특별시 양천구 가로공원로 71,5호선 마곡,1658.491978,https://place.map.kakao.com/1192328262,마곡,마곡역 리버스로스터
61,필링,커피점/카페,커피전문점/카페/다방,서울특별시 종로구 수표로 110,5호선 종로3가(탑골공원),125.480924,https://place.map.kakao.com/13056584,종로3가,종로3가역 필링
...,...,...,...,...,...,...,...,...,...
20025,세라젬웰카페,커피점/카페,커피전문점/카페/다방,서울특별시 영등포구 영등포로 210,5호선 영등포시장,215.460909,NaN,영등포시장,영등포시장역 세라젬웰카페
20026,미타임,커피점/카페,커피전문점/카페/다방,서울특별시 성동구 천호대로 346,5호선 답십리,587.455177,NaN,답십리,답십리역 미타임
20027,휴식상점,커피점/카페,커피전문점/카페/다방,서울특별시 동대문구 한천로6길 47-2,5호선 장한평,290.438891,NaN,장한평,장한평역 휴식상점
20043,단백한하우스,커피점/카페,커피전문점/카페/다방,서울특별시 송파구 문정로11길 12,5호선 개롱,357.124389,NaN,개롱,개롱역 단백한하우스


In [11]:
img_elem

,ItemID,img_url
0,양평역 매머드커피익스프레스,None
1,양평역 매머드커피익스프레스,None
2,양평역 매머드커피익스프레스,None
3,양평역 매머드커피익스프레스,None
4,마포역 일야에스프레소,None
...,...,...
7640,화곡역 메리하하,None
7641,화곡역 메리하하,None
7642,굽은다리역 카페1046,None
7643,신정역 카페펀코,None


In [12]:
rating_df

,UserID,ItemID,Rating,Timestamp
0,cj0olb,양평역 매머드커피익스프레스,2점,2019.12.01.
1,mlm7f0,양평역 매머드커피익스프레스,5점,2019.10.20.
2,vdui22,양평역 매머드커피익스프레스,5점,2019.08.26.
3,u9keii,양평역 매머드커피익스프레스,4점,2019.08.23.
4,vaui89,마포역 일야에스프레소,1점,2021.05.05.
...,...,...,...,...
7640,None,화곡역 메리하하,None,2018.12.12.
7641,,화곡역 메리하하,5점,2018.08.10.
7642,None,굽은다리역 카페1046,None,2016.12.15.
7643,None,신정역 카페펀코,None,2016.06.28.


In [13]:
review_elem

,ItemID,review
0,양평역 매머드커피익스프레스,녹차 샷 시켰는데 녹차 가루가 다 안 섞여서 둥둥 떠다니네요
1,양평역 매머드커피익스프레스,싸고 친절하고 빨리나오고 깨끗하고 굿굿
2,양평역 매머드커피익스프레스,테이크아웃할거면 여기서 사먹는게 제일나음. 여기 먹다보니까 다른곳 못먹겠음.
3,양평역 매머드커피익스프레스,"타 지점의 가격보다도 싼 아메리카노(L), 맛도 나쁘지 않았습니다. 7시에 오픈~8..."
4,마포역 일야에스프레소,연희동 공간커피 처럼 주인장 남자분 너무 매섭게 노트북 여는 즉시 1시간...
...,...,...
7640,화곡역 메리하하,최고예요
7641,화곡역 메리하하,화곡에서 가장 맛있는 샌드위치입니다. 정말 맛이 끝내줍니다.
7642,굽은다리역 카페1046,좋아요
7643,신정역 카페펀코,작은 까페지만 커피맛 괜찮아요


In [14]:
img_elem.head()

,ItemID,img_url
0,양평역 매머드커피익스프레스,None
1,양평역 매머드커피익스프레스,None
2,양평역 매머드커피익스프레스,None
3,양평역 매머드커피익스프레스,None
4,마포역 일야에스프레소,None


In [16]:
ItemID_elem = rating_df['ItemID'].unique()

for k in range(len(ItemID_elem)) :
    User_ID_elem = rating_df.loc[rating_df['ItemID'] == ItemID_elem[k]]
    Uder_ID_elem = User_ID_elem[['UserID','ItemID']]
print(User_ID_elem)





ItemID_elem = img_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_img_elem = img_elem.loc[img_elem['ItemID'] == ItemID_elem[j]]
print(item_img_elem)



ItemID_elem = review_elem['ItemID'].unique()
for j in range(len(ItemID_elem)) :
    item_review_elem = review_elem.loc[review_elem['ItemID'] == ItemID_elem[j]]
print(item_review_elem)




      UserID       ItemID Rating    Timestamp
1023  7m278d  영등포시장역 스타벅스     5점  2021.05.01.
1024    None  영등포시장역 스타벅스   None  2021.04.01.
           ItemID img_url
1023  영등포시장역 스타벅스    None
           ItemID review
1023  영등포시장역 스타벅스   최고예요


In [17]:
rating_df.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe(2255).csv')

In [18]:
User_ID_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe_User(2255).csv')
item_img_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_cafe_img(2255).csv')
item_review_elem.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/lline5_cafe_review(2255).csv')

In [15]:
ItemID_elem = rating_df['ItemID'].unique()
len(ItemID_elem)


902

In [16]:
data.to_csv('C:/Users/정예림/Desktop/추천시스템 프로젝트/data/line5_(1)2255url.csv')